# Example 14: Integration Testing Basics

## Learning Objective
Learn to write integration tests that verify multiple components work together.

---

## Unit vs Integration Tests

| Unit Tests | Integration Tests |
|------------|-------------------|
| Test single function | Test multiple components |
| Use mocks | Use real (or realistic) dependencies |
| Very fast | Slower |
| Isolate bugs precisely | Catch interaction bugs |

In [ ]:
# Simple todo system with multiple layers

class TodoRepository:
    """Data layer."""
    def __init__(self):
        self._todos = {}
        self._next_id = 1
    
    def add(self, title):
        todo = {"id": self._next_id, "title": title, "done": False}
        self._todos[self._next_id] = todo
        self._next_id += 1
        return todo
    
    def get(self, todo_id):
        return self._todos.get(todo_id)
    
    def get_all(self):
        return list(self._todos.values())
    
    def update(self, todo_id, **kwargs):
        if todo_id in self._todos:
            self._todos[todo_id].update(kwargs)
            return True
        return False


class TodoService:
    """Business logic layer."""
    def __init__(self, repo):
        self.repo = repo
    
    def create_todo(self, title):
        if not title.strip():
            raise ValueError("Title required")
        return self.repo.add(title.strip())
    
    def complete_todo(self, todo_id):
        todo = self.repo.get(todo_id)
        if not todo:
            return None
        self.repo.update(todo_id, done=True)
        return self.repo.get(todo_id)
    
    def get_pending(self):
        return [t for t in self.repo.get_all() if not t["done"]]

## Integration Tests

In [ ]:
def test_create_and_complete_workflow():
    """Test the complete todo workflow."""
    # Setup - real components working together
    repo = TodoRepository()
    service = TodoService(repo)
    
    # Create todos
    todo1 = service.create_todo("Task 1")
    todo2 = service.create_todo("Task 2")
    todo3 = service.create_todo("Task 3")
    
    # Verify all pending
    assert len(service.get_pending()) == 3
    
    # Complete one
    completed = service.complete_todo(todo2["id"])
    assert completed["done"] == True
    
    # Verify pending count
    pending = service.get_pending()
    assert len(pending) == 2
    assert all(t["id"] != todo2["id"] for t in pending)
    
    print("✓ test_create_and_complete_workflow passed")


def test_complete_nonexistent():
    """Test completing a todo that doesn't exist."""
    repo = TodoRepository()
    service = TodoService(repo)
    
    result = service.complete_todo(999)
    
    assert result is None
    print("✓ test_complete_nonexistent passed")


def test_validation_integration():
    """Test that validation works through the layers."""
    repo = TodoRepository()
    service = TodoService(repo)
    
    try:
        service.create_todo("   ")  # Empty after strip
        assert False, "Should have raised"
    except ValueError:
        pass
    
    print("✓ test_validation_integration passed")


# Run integration tests
test_create_and_complete_workflow()
test_complete_nonexistent()
test_validation_integration()

## When to Use Integration Tests

- Testing database interactions
- Testing API endpoints
- Testing service-to-service communication
- Verifying configuration
- Testing complex workflows

In [ ]:
# Practice: Add more integration tests
